## Create Relative Landmark Coordinates by subtracting Hand Movement Offset, calculating the Difference from Reference Pose and Normalizing

This script processes 3D kinematic data in absolute pixel cooridnates for multiple participants and matches to create hand movements relative to the wrist's position and the initial posture during rest. This involves shifting the coordinate system to the wrist and tracking the movement of each landmark (e.g., fingertips) relative to their starting positions.

- **Low-Pass Filtering:** Removes high-frequency noise using a Butterworth filter.
- **Wrist-Pinning & Subtraction:** Centers the wrist at the origin and subtracts a reference posture to focus on relative motion.
- **Distance Computation:** Calculates Euclidean distances between specified landmarks (e.g., wrist to fingertip).
- **Normalization:** Scales data amplitudes for consistency.
- **Saving & Visualization:** Saves processed data and generates comparison plots (before vs. after processing).
- **Execution:** It loops through participants (P1–P8) and matches, processing .npy files, saving normalized outputs, and plotting results.

In [ ]:
#!/usr/bin/env python3
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# ------------------------------------------------------------------------------------
# 1) Low-Pass Filtering Utility
# ------------------------------------------------------------------------------------
def lowpass_filter_3D(data_3D, cutoff_hz, fs, order=4):
    """
    Applies a zero-phase Butterworth low-pass filter to each column (channel)
    in data_3D using scipy's filtfilt. This removes high-frequency noise.

    Parameters
    ----------
    data_3D : np.ndarray, shape (num_frames, num_cols)
        Each column is one channel, e.g. (x of L0, y of L0, z of L0, x of L1, etc.)
    cutoff_hz : float
        Cutoff frequency in Hz (must be < fs/2).
    fs : float
        Sampling frequency in Hz, e.g. 30.0 if 30 samples per second.
    order : int
        Filter order (e.g., 4).

    Returns
    -------
    filtered : np.ndarray, same shape as data_3D
        The smoothed version of data_3D.
    """
    nyquist = 0.5 * fs
    Wn = cutoff_hz / nyquist
    b, a = butter(order, Wn, btype='low', analog=False)
    filtered = np.zeros_like(data_3D)

    for col_idx in range(data_3D.shape[1]):
        filtered[:, col_idx] = filtfilt(b, a, data_3D[:, col_idx])

    return filtered

# ------------------------------------------------------------------------------------
# 2) Relative Kinematic Data (Wrist-Pinning & Subtraction)
# ------------------------------------------------------------------------------------
def create_relative_kinematic_data(phase_kin, ref_local_posture, n_total_landmarks=23, wrist_idx=0):
    """
    1) Pins the chosen wrist landmark at (0,0,0) for every frame.
    2) Subtracts the rest posture from the first frame (i0).
    """
    num_frames = phase_kin.shape[0]
    # Reshape => (num_frames, n_total_landmarks, 3)
    kin_reshaped = phase_kin.reshape(num_frames, n_total_landmarks, 3)

    # Wrist positions => (num_frames, 3)
    wrist_positions = kin_reshaped[:, wrist_idx, :]

    # Pin the wrist => local positions
    local_positions = kin_reshaped - wrist_positions[:, np.newaxis, :]

    # Subtract the rest posture => focusing on relative changes
    kin_relative = local_positions - ref_local_posture[np.newaxis, :, :]

    # Reshape back => (num_frames, 3 * n_total_landmarks)
    return kin_relative.reshape(num_frames, n_total_landmarks*3)

# ------------------------------------------------------------------------------------
# 3) Distance Computation
# ------------------------------------------------------------------------------------
def compute_distance(kin_data, idxA=0, idxB=8):
    """
    Computes Euclidean distance between landmarks idxA and idxB for each frame.
    kin_data shape => (num_frames, 3*n_total_landmarks).
    """
    num_frames = kin_data.shape[0]
    distances = np.zeros(num_frames, dtype=np.float32)

    for i in range(num_frames):
        Ax = kin_data[i, idxA*3 + 0]
        Ay = kin_data[i, idxA*3 + 1]
        Az = kin_data[i, idxA*3 + 2]
        Bx = kin_data[i, idxB*3 + 0]
        By = kin_data[i, idxB*3 + 1]
        Bz = kin_data[i, idxB*3 + 2]
        dx, dy, dz = (Bx - Ax), (By - Ay), (Bz - Az)
        distances[i] = np.sqrt(dx*dx + dy*dy + dz*dz)

    return distances

# ------------------------------------------------------------------------------------
# 4) Plot (4 Subplots => Phase1-Before, Phase1-After, Phase2-Before, Phase2-After)
# ------------------------------------------------------------------------------------
def plot_comparison(
    phase1_before, phase1_after,
    phase2_before, phase2_after,
    pnum, match_prefix,
    idxA=0,  # wrist
    idxB=8   # fingertip
):
    """
    Draws a 2x2 subplot grid:
      - Phase1 BEFORE (top-left)
      - Phase1 AFTER  (top-right)
      - Phase2 BEFORE (bottom-left)
      - Phase2 AFTER  (bottom-right)

    Each subplot has its own y-scale so that small 'AFTER' data
    isn't dwarfed by large 'BEFORE' data.
    """
    dist_p1_before = compute_distance(phase1_before, idxA, idxB)
    dist_p1_after  = compute_distance(phase1_after,  idxA, idxB)
    frames_p1 = np.arange(len(dist_p1_before))

    dist_p2_before = compute_distance(phase2_before, idxA, idxB)
    dist_p2_after  = compute_distance(phase2_after,  idxA, idxB)
    frames_p2 = np.arange(len(dist_p2_before))

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Phase1 BEFORE => top-left
    axs[0, 0].plot(frames_p1, dist_p1_before, color="red")
    axs[0, 0].set_title(f"P{pnum}, {match_prefix}\nPhase1 BEFORE: {idxA}→{idxB}")
    axs[0, 0].set_xlabel("Frame")
    axs[0, 0].set_ylabel("Distance")

    # Phase1 AFTER => top-right
    axs[0, 1].plot(frames_p1, dist_p1_after, color="blue")
    axs[0, 1].set_title(f"P{pnum}, {match_prefix}\nPhase1 AFTER: {idxA}→{idxB}")
    axs[0, 1].set_xlabel("Frame")
    axs[0, 1].set_ylabel("Distance")

    # Phase2 BEFORE => bottom-left
    axs[1, 0].plot(frames_p2, dist_p2_before, color="red")
    axs[1, 0].set_title(f"P{pnum}, {match_prefix}\nPhase2 BEFORE: {idxA}→{idxB}")
    axs[1, 0].set_xlabel("Frame")
    axs[1, 0].set_ylabel("Distance")

    # Phase2 AFTER => bottom-right
    axs[1, 1].plot(frames_p2, dist_p2_after, color="blue")
    axs[1, 1].set_title(f"P{pnum}, {match_prefix}\nPhase2 AFTER: {idxA}→{idxB}")
    axs[1, 1].set_xlabel("Frame")
    axs[1, 1].set_ylabel("Distance")

    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------------
# 5) Main Pipeline
# ------------------------------------------------------------------------------------
def main():
    """
    Main pipeline:
      - Loops over participants P(1)..P(8) and matches match_01..match_24
      - Loads *_phase1_kin.npy, *_phase2_kin.npy
      - Filters them (low-pass) BEFORE wrist-pinning
      - Wrist-pins & subtracts rest posture
      - Normalizes
      - Saves results
      - Plots 4-subplot comparison
    """
    base_dir = r"C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data"

    # Data has 21 hand + 2 pose => 23 total => 69 columns
    n_total_landmarks = 23
    wrist_idx = 0   # The hand wrist is landmark #0 
    tip_idx   = 8   # Index finger tip in MediaPipe hand

    # Filter settings
    fs = 30.0      
    cutoff_hz = 1.0 
    filter_order = 4

    for pnum in range(1, 9):
        participant_str = f"P({pnum})"
        phase_dir = os.path.join(base_dir, participant_str, "Synchronized Data split in Phases")
        if not os.path.isdir(phase_dir):
            print(f"[WARN] No phase dir => {phase_dir}, skipping P({pnum})")
            continue

        print(f"\n[INFO] === Processing Participant {participant_str} ===")
        for m in range(1, 25):
            match_prefix = f"match_{m:02d}"
            phase1_kin_path = os.path.join(phase_dir, f"{match_prefix}_phase1_kin.npy")
            phase2_kin_path = os.path.join(phase_dir, f"{match_prefix}_phase2_kin.npy")

            if not (os.path.isfile(phase1_kin_path) and os.path.isfile(phase2_kin_path)):
                continue

            print(f"[MATCH] {match_prefix} => Loading Phase1 & Phase2")
            phase1_kin_original = np.load(phase1_kin_path)
            phase2_kin_original = np.load(phase2_kin_path)

            if phase1_kin_original.shape[0] < 1:
                print(f"[WARN] Phase1 {match_prefix} has 0 frames, skip.")
                continue

            expected_cols = n_total_landmarks * 3
            if phase1_kin_original.shape[1] != expected_cols:
                print(f"[WARN] Phase1 => {phase1_kin_original.shape[1]} cols, expected {expected_cols}")
            if phase2_kin_original.shape[1] != expected_cols:
                print(f"[WARN] Phase2 => {phase2_kin_original.shape[1]} cols, expected {expected_cols}")

            # ------------------------------------------------------------------
            # (A) FILTER FIRST: remove high-freq noise
            # ------------------------------------------------------------------
            phase1_filt = lowpass_filter_3D(phase1_kin_original, cutoff_hz, fs, order=filter_order)
            phase2_filt = lowpass_filter_3D(phase2_kin_original, cutoff_hz, fs, order=filter_order)

            # ------------------------------------------------------------------
            # (B) Pin wrist & subtract rest posture from Phase1's first frame
            # ------------------------------------------------------------------
            i0_frame = phase1_filt[0].reshape(n_total_landmarks, 3)
            i0_wrist = i0_frame[wrist_idx]
            ref_local_posture = i0_frame - i0_wrist

            phase1_relative = create_relative_kinematic_data(
                phase_kin=phase1_filt,
                ref_local_posture=ref_local_posture,
                n_total_landmarks=n_total_landmarks,
                wrist_idx=wrist_idx
            )
            phase2_relative = create_relative_kinematic_data(
                phase_kin=phase2_filt,
                ref_local_posture=ref_local_posture,
                n_total_landmarks=n_total_landmarks,
                wrist_idx=wrist_idx
            )

            # ------------------------------------------------------------------
            # (C) Amplitude Normalization
            # ------------------------------------------------------------------
            max_val_1 = np.abs(phase1_relative).max() if phase1_relative.size else 0.0
            max_val_2 = np.abs(phase2_relative).max() if phase2_relative.size else 0.0
            global_max = max(max_val_1, max_val_2)
            if global_max < 1e-12:
                global_max = 1.0
                print(f"[WARN] No motion => skipping amplitude scaling for {match_prefix}.")

            phase1_norm = phase1_relative / global_max
            phase2_norm = phase2_relative / global_max

            # ------------------------------------------------------------------
            # (D) Save
            # ------------------------------------------------------------------
            out_phase1 = os.path.join(phase_dir, f"{match_prefix}_phase1_kin_norm.npy")
            out_phase2 = os.path.join(phase_dir, f"{match_prefix}_phase2_kin_norm.npy")
            np.save(out_phase1, phase1_norm)
            np.save(out_phase2, phase2_norm)

            print(f"[SAVED] => {out_phase1}")
            print(f"[SAVED] => {out_phase2}")

            # ------------------------------------------------------------------
            # (E) Plot 4-subplot comparison
            # ------------------------------------------------------------------
            plot_comparison(
                phase1_before=phase1_kin_original,
                phase1_after=phase1_norm,
                phase2_before=phase2_kin_original,
                phase2_after=phase2_norm,
                pnum=pnum,
                match_prefix=match_prefix,
                idxA=wrist_idx,
                idxB=tip_idx
            )

    print("\n[ALL DONE] Smoothing + Wrist-Pinning + Normalization done for P(1)..P(8).")

if __name__ == "__main__":
    main()
